In [1]:
import pandas as pd
import numpy as np
import glob
import gc
import keras 
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate
from keras.models import Model
import random
from sklearn.model_selection import KFold
import keras.backend as K
from sklearn.utils import class_weight

Using TensorFlow backend.


In [2]:
gc.enable()

In [3]:
steps = 5
height = 108
width = 64
width_prev = 256

In [4]:
#change the path to where you save the input numpy arrays
prev_cnn = np.array(glob.glob('E:/notes_database/CNN_previous/*'))
cnn = np.array(glob.glob('E:/notes_database/pureCNN/*'))
y_train = np.array(glob.glob('E:/notes_database/y_pureCNN/*'))

In [5]:
#The function analyzes the actual accuracy
def load_test_score(model, cnn_test, prev_cnn_test, y_test):
    prediction = model.predict([cnn_test, prev_cnn_test])
    error = (prediction - y_test) ** 2
    error = (error/y_test.shape[0]).sum()
    wrong = 0
    for x in range(prediction.shape[0]):
        for y in range(prediction.shape[1]):
            if abs(prediction[x][y] - y_test[x][y]) > 0.5:
                wrong += 1
                break
    accuracy = 1 - wrong/(prediction.shape[0])
    return error, accuracy

In [6]:
def inputs_to_model(order, num, start, ending):
    cnn_inputs = np.load(cnn[order[num][0]])
    labels_inputs = np.load(y_train[order[num][0]])
    prevcnn_inputs = np.load(prev_cnn[order[num][0]])
    
    count = 0
    ce = 0
    for file in order[num][start+1:ending]:
        if count == ce + 50:
            print(count)
            ce = count
        count+=1
        
        prevcnn_inputs = np.append(prevcnn_inputs, np.load(prev_cnn[file]), axis=0)
        cnn_inputs = np.append(cnn_inputs, np.load(cnn[file]), axis=0)
        labels_inputs = np.append(labels_inputs, np.load(y_train[file]), axis=0)
        
    return prevcnn_inputs, cnn_inputs, labels_inputs

In [7]:
def main(optimizer='adam', loss='mean_squared_error', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(16, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.2)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #prev_cnn
    prevcnn_inputs = Input(shape=(height, width_prev, 1), name='prevcnn_inputs')
    prev_layers = Conv2D(32, (3,3), padding="same", activation="relu")(prevcnn_inputs)
    prev_layers = BatchNormalization()(prev_layers)
    prev_layers = MaxPooling2D(3,3, border_mode='same')(prev_layers)
    prev_layers = Conv2D(64, (3,3), activation='relu')(prev_layers)
    prev_layers = BatchNormalization()(prev_layers)
    prev_layers = MaxPooling2D(2,2, border_mode='same')(prev_layers)
    prev_layers = Conv2D(64, (3,3), activation='relu')(prev_layers)
    prev_layers = BatchNormalization()(prev_layers)
    prev_layers = MaxPooling2D(2,2, border_mode='same')(prev_layers)
    prev_layers = Conv2D(64, (3,3), activation='relu')(prev_layers)
    prev_layers = BatchNormalization()(prev_layers)
    prev_layers = MaxPooling2D(2,2, border_mode='same')(prev_layers)
    prev_layers = Flatten()(prev_layers)
    prev_layers = Dropout(0.3)(prev_layers)
    prev_layers = Dense(64, activation='relu')(prev_layers)
    
    #main route
    main_ = concatenate([layers, prev_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, prevcnn_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [8]:
def main_2(optimizer='adam', loss='mean_squared_error', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(16, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.2)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #prev_cnn
    prevcnn_inputs = Input(shape=(height, width_prev, 1), name='prevcnn_inputs')
    prev_layers = Conv2D(32, (3,3), padding="same", activation="relu")(prevcnn_inputs)
    prev_layers = BatchNormalization()(prev_layers)
    prev_layers = MaxPooling2D(3,3, border_mode='same')(prev_layers)
    prev_layers = Conv2D(64, (3,3), activation='relu')(prev_layers)
    prev_layers = BatchNormalization()(prev_layers)
    prev_layers = MaxPooling2D(2,2, border_mode='same')(prev_layers)
    prev_layers = Conv2D(32, (3,3), activation='relu')(prev_layers)
    prev_layers = BatchNormalization()(prev_layers)
    prev_layers = MaxPooling2D(2,2, border_mode='same')(prev_layers)
    prev_layers = Flatten()(prev_layers)
    prev_layers = Dropout(0.3)(prev_layers)
    prev_layers = Dense(64, activation='relu')(prev_layers)
    
    #main route
    main_ = concatenate([layers, prev_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, prevcnn_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [9]:
def train_test_split(classifier, num, name, epochs=100, batchsize=32, optimizer='adam', loss='mean_squared_error',
                     metrix='accuracy', split=2):
    for epoch in range(epochs):
        print("""
        
        THIS IS EPOCH {}""".format(epoch))
        for i in range(split):
            prev,cnn,labels = inputs_to_model(train_order, num, (train_order[num].shape[0]*i)//split, 
                                              (train_order[num].shape[0]*(i+1))//split+1)
            
            classifier.fit([cnn, prev], labels, epochs=1, batch_size=batchsize, verbose=1,
                  validation_data = ([cnn_test, prevcnn_test], labels_test))
            
            # To save memory, it's possible to train the model first then test on the validation data.
            #CV_loss, accuracy = load_test_score(classifier, cnn_test, lstm_test, labels_test)
            #print('val_loss: ', CV_loss)
            #print('val_accuarcy', accuracy)
    
    classifier.save_weights(name, overwrite=True)
    del classifier
    K.clear_session()

In [10]:
#Generate the CV sets
kf = KFold(n_splits=5, random_state=7)

train_order = []
test_order = []
length = np.arange(len(cnn))

for x,y in kf.split(length):
    train_order.append(length[x])
    test_order.append(length[y])
train_order = np.array(train_order)
test_order = np.array(test_order)

In [11]:
prevcnn_test, cnn_test, labels_test = inputs_to_model(test_order, 0, 0, test_order[0].shape[0]+1)

In [12]:
# without sample_weight
adam = keras.optimizers.Adam(lr=0.005)
classifier = main(optimizer=adam, loss='binary_crossentropy')
train_test_split(classifier, 0, 'weight-split-simpler-purecnn.hdf5', epochs=100, split=3)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if sys.path[0] == '':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(3, 3, padding="same")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding=


        
        THIS IS EPOCH 0
50
Train on 29967 samples, validate on 22321 samples
Epoch 1/1
29967/29967 [==============================] - 59s 2ms/step - loss: 0.1064 - acc: 0.9614 - val_loss: 0.0421 - val_acc: 0.9863
50
Train on 29799 samples, validate on 22321 samples
Epoch 1/1
29799/29799 [==============================] - 55s 2ms/step - loss: 0.0413 - acc: 0.9864 - val_loss: 0.0280 - val_acc: 0.9909
50
Train on 29930 samples, validate on 22321 samples
Epoch 1/1
29930/29930 [==============================] - 55s 2ms/step - loss: 0.0346 - acc: 0.9889 - val_loss: 0.0260 - val_acc: 0.9917

        
        THIS IS EPOCH 1
50
Train on 29967 samples, validate on 22321 samples
Epoch 1/1
29967/29967 [==============================] - 55s 2ms/step - loss: 0.0298 - acc: 0.9906 - val_loss: 0.0226 - val_acc: 0.9930
50
Train on 29799 samples, validate on 22321 samples
Epoch 1/1
29799/29799 [==============================] - 55s 2ms/step - loss: 0.0278 - acc: 0.9913 - val_loss: 0.0180 - val

50
Train on 29930 samples, validate on 22321 samples
Epoch 1/1
29930/29930 [==============================] - 55s 2ms/step - loss: 0.0129 - acc: 0.9961 - val_loss: 0.0104 - val_acc: 0.9972

        
        THIS IS EPOCH 14
50
Train on 29967 samples, validate on 22321 samples
Epoch 1/1
29967/29967 [==============================] - 55s 2ms/step - loss: 0.0121 - acc: 0.9964 - val_loss: 0.0162 - val_acc: 0.9958
50
Train on 29799 samples, validate on 22321 samples
Epoch 1/1
29799/29799 [==============================] - 57s 2ms/step - loss: 0.0120 - acc: 0.9963 - val_loss: 0.0118 - val_acc: 0.9969
50
Train on 29930 samples, validate on 22321 samples
Epoch 1/1
29930/29930 [==============================] - 57s 2ms/step - loss: 0.0123 - acc: 0.9963 - val_loss: 0.0135 - val_acc: 0.9966

        
        THIS IS EPOCH 15
50
Train on 29967 samples, validate on 22321 samples
Epoch 1/1
29967/29967 [==============================] - 57s 2ms/step - loss: 0.0120 - acc: 0.9964 - val_loss: 0.0123 - v

29930/29930 [==============================] - 55s 2ms/step - loss: 0.0089 - acc: 0.9974 - val_loss: 0.0090 - val_acc: 0.9978

        
        THIS IS EPOCH 41
50
Train on 29967 samples, validate on 22321 samples
Epoch 1/1
29967/29967 [==============================] - 55s 2ms/step - loss: 0.0086 - acc: 0.9975 - val_loss: 0.0128 - val_acc: 0.9972
50
Train on 29799 samples, validate on 22321 samples
Epoch 1/1
29799/29799 [==============================] - 55s 2ms/step - loss: 0.0084 - acc: 0.9975 - val_loss: 0.0117 - val_acc: 0.9972
50
Train on 29930 samples, validate on 22321 samples
Epoch 1/1
29930/29930 [==============================] - 56s 2ms/step - loss: 0.0085 - acc: 0.9975 - val_loss: 0.0114 - val_acc: 0.9976

        
        THIS IS EPOCH 42
50
Train on 29967 samples, validate on 22321 samples
Epoch 1/1
29967/29967 [==============================] - 55s 2ms/step - loss: 0.0088 - acc: 0.9974 - val_loss: 0.0119 - val_acc: 0.9968
50
Train on 29799 samples, validate on 22321 sam

29799/29799 [==============================] - 55s 2ms/step - loss: 0.0077 - acc: 0.9976 - val_loss: 0.0135 - val_acc: 0.9972
50
Train on 29930 samples, validate on 22321 samples
Epoch 1/1
29930/29930 [==============================] - 55s 2ms/step - loss: 0.0081 - acc: 0.9976 - val_loss: 0.0093 - val_acc: 0.9978

        
        THIS IS EPOCH 55
50
Train on 29967 samples, validate on 22321 samples
Epoch 1/1
29967/29967 [==============================] - 55s 2ms/step - loss: 0.0079 - acc: 0.9977 - val_loss: 0.0090 - val_acc: 0.9978
50
Train on 29799 samples, validate on 22321 samples
Epoch 1/1
29799/29799 [==============================] - 55s 2ms/step - loss: 0.0073 - acc: 0.9978 - val_loss: 0.0104 - val_acc: 0.9977
50
Train on 29930 samples, validate on 22321 samples
Epoch 1/1
29930/29930 [==============================] - 54s 2ms/step - loss: 0.0078 - acc: 0.9977 - val_loss: 0.0108 - val_acc: 0.9974

        
        THIS IS EPOCH 56
50
Train on 29967 samples, validate on 22321 sam

29967/29967 [==============================] - 56s 2ms/step - loss: 0.0071 - acc: 0.9978 - val_loss: 0.0127 - val_acc: 0.9975
50
Train on 29799 samples, validate on 22321 samples
Epoch 1/1
29799/29799 [==============================] - 55s 2ms/step - loss: 0.0072 - acc: 0.9978 - val_loss: 0.0100 - val_acc: 0.9978
50
Train on 29930 samples, validate on 22321 samples
Epoch 1/1
29930/29930 [==============================] - 56s 2ms/step - loss: 0.0074 - acc: 0.9978 - val_loss: 0.0119 - val_acc: 0.9977

        
        THIS IS EPOCH 69
50
Train on 29967 samples, validate on 22321 samples
Epoch 1/1
29967/29967 [==============================] - 55s 2ms/step - loss: 0.0069 - acc: 0.9979 - val_loss: 0.0132 - val_acc: 0.9974
50
Train on 29799 samples, validate on 22321 samples
Epoch 1/1
29799/29799 [==============================] - 55s 2ms/step - loss: 0.0070 - acc: 0.9979 - val_loss: 0.0105 - val_acc: 0.9976
50
Train on 29930 samples, validate on 22321 samples
Epoch 1/1
29930/29930 [=======

29930/29930 [==============================] - 58s 2ms/step - loss: 0.0075 - acc: 0.9978 - val_loss: 0.0112 - val_acc: 0.9976

        
        THIS IS EPOCH 82
50
Train on 29967 samples, validate on 22321 samples
Epoch 1/1
29967/29967 [==============================] - 58s 2ms/step - loss: 0.0068 - acc: 0.9980 - val_loss: 0.0097 - val_acc: 0.9977
50
Train on 29799 samples, validate on 22321 samples
Epoch 1/1
29799/29799 [==============================] - 57s 2ms/step - loss: 0.0069 - acc: 0.9979 - val_loss: 0.0109 - val_acc: 0.9976
50
Train on 29930 samples, validate on 22321 samples
Epoch 1/1
29930/29930 [==============================] - 57s 2ms/step - loss: 0.0068 - acc: 0.9979 - val_loss: 0.0096 - val_acc: 0.9977

        
        THIS IS EPOCH 83
50
Train on 29967 samples, validate on 22321 samples
Epoch 1/1
29967/29967 [==============================] - 57s 2ms/step - loss: 0.0066 - acc: 0.9981 - val_loss: 0.0107 - val_acc: 0.9976
50
Train on 29799 samples, validate on 22321 sam

29799/29799 [==============================] - 63s 2ms/step - loss: 0.0069 - acc: 0.9980 - val_loss: 0.0107 - val_acc: 0.9977
50
Train on 29930 samples, validate on 22321 samples
Epoch 1/1
29930/29930 [==============================] - 63s 2ms/step - loss: 0.0064 - acc: 0.9981 - val_loss: 0.0136 - val_acc: 0.9974

        
        THIS IS EPOCH 96
50
Train on 29967 samples, validate on 22321 samples
Epoch 1/1
29967/29967 [==============================] - 63s 2ms/step - loss: 0.0064 - acc: 0.9981 - val_loss: 0.0120 - val_acc: 0.9976
50
Train on 29799 samples, validate on 22321 samples
Epoch 1/1
29799/29799 [==============================] - 61s 2ms/step - loss: 0.0065 - acc: 0.9980 - val_loss: 0.0131 - val_acc: 0.9970
50
Train on 29930 samples, validate on 22321 samples
Epoch 1/1
29930/29930 [==============================] - 55s 2ms/step - loss: 0.0068 - acc: 0.9980 - val_loss: 0.0114 - val_acc: 0.9977

        
        THIS IS EPOCH 97
50
Train on 29967 samples, validate on 22321 sam

39294

In [13]:
classifier = main(loss='binary_crossentropy')
classifier.load_weights('weight-split-simpler-purecnn.hdf5')

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if sys.path[0] == '':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(3, 3, padding="same")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding=

In [14]:
load_test_score(classifier, cnn_test, prevcnn_test, labels_test)

(0.046346884, 0.9475381927333004)